In [15]:
# import libraries
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from torch.utils.data import DataLoader
from torchvision import datasets, transforms

import sys
from tqdm import tqdm


import matplotlib.pyplot as plt
import matplotlib_inline.backend_inline
matplotlib_inline.backend_inline.set_matplotlib_formats('svg')

In [ ]:
print("CUDA available:", torch.cuda.is_available())  
print("CUDA version:", torch.version.cuda)          
print("Device:", torch.device("cuda"))             
print("GPU name:", torch.cuda.get_device_name(0))

CUDA available: True
CUDA version: 13.0
Device: cuda
GPU name: NVIDIA GeForce RTX 3060


In [ ]:
transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
    transforms.Normalize([0.5,0.5,0.5], [0.5,0.5,0.5])
])

dataset = datasets.ImageFolder(root='Faulty_solar_panel', transform=transform)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True, pin_memory=True)


In [ ]:
print("Classes:", dataset.classes)
num_classes = len(dataset.classes)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)

Klasy: ['Bird-drop', 'Clean', 'Dusty', 'Electrical-damage', 'Physical-Damage', 'Snow-Covered']
Device: cuda


In [19]:
class Autoencoder(nn.Module):
    def __init__(self):
        super(Autoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Conv2d(3, 16, 3, stride=2, padding=1),  # [3,256,256] -> [16,128,128]
            nn.ReLU(),
            nn.Conv2d(16,32,3,stride=2,padding=1),    # [16,128,128] -> [32,64,64]
            nn.ReLU(),
            nn.Conv2d(32,64,3,stride=2,padding=1),    # [32,64,64] -> [64,32,32]
            nn.ReLU(),
            nn.Conv2d(64,128,3,stride=2,padding=1),   # [64,32,32] -> [128,16,16]
            nn.ReLU()
        )
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(128,64,3,stride=2,padding=1,output_padding=1),
            nn.ReLU(),
            nn.ConvTranspose2d(64,32,3,stride=2,padding=1,output_padding=1),
            nn.ReLU(),
            nn.ConvTranspose2d(32,16,3,stride=2,padding=1,output_padding=1),
            nn.ReLU(),
            nn.ConvTranspose2d(16,3,3,stride=2,padding=1,output_padding=1),
            nn.Sigmoid()
        )

    def forward(self,x):
        latent = self.encoder(x)
        recon = self.decoder(latent)
        return recon

autoencoder = Autoencoder().to(device)


In [20]:
ae_criterion = nn.MSELoss()
ae_optimizer = optim.Adam(autoencoder.parameters(), lr=1e-3)
num_ae_epochs = 100

print("Pretraining autoencoder...")
for epoch in range(num_ae_epochs):
    running_loss = 0.0
    for images, _ in tqdm(dataloader):
        images = images.to(device)
        outputs = autoencoder(images)
        loss = ae_criterion(outputs, images)

        ae_optimizer.zero_grad()
        loss.backward()
        ae_optimizer.step()

        running_loss += loss.item() * images.size(0)

    epoch_loss = running_loss / len(dataset)
    print(f"AE Epoch [{epoch+1}/{num_ae_epochs}], Loss: {epoch_loss:.6f}")


Pretraining autoencoder...


100%|██████████| 28/28 [00:12<00:00,  2.19it/s]


AE Epoch [1/100], Loss: 0.434057


100%|██████████| 28/28 [00:12<00:00,  2.19it/s]


AE Epoch [2/100], Loss: 0.197010


...

AE Epoch [98/100], Loss: 0.140065


100%|██████████| 28/28 [00:13<00:00,  2.07it/s]


AE Epoch [99/100], Loss: 0.140227


100%|██████████| 28/28 [00:13<00:00,  2.08it/s]

AE Epoch [100/100], Loss: 0.139992


In [21]:
class Classifier(nn.Module):
    def __init__(self, latent_channels=128, num_classes=3):
        super(Classifier, self).__init__()
        self.pool = nn.AdaptiveAvgPool2d((1,1))
        self.fc = nn.Sequential(
            nn.Flatten(),
            nn.Linear(latent_channels, 64),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(64, num_classes)
        )
    def forward(self,x):
        x = self.pool(x)
        x = self.fc(x)
        return x

classifier = Classifier(latent_channels=128, num_classes=num_classes).to(device)

In [22]:
for param in autoencoder.encoder.parameters():
    param.requires_grad = True

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(list(autoencoder.encoder.parameters()) + list(classifier.parameters()), lr=1e-4)

In [23]:
num_epochs = 100
print("Training classifier with encoder fine-tuning...")
for epoch in range(num_epochs):
    running_loss = 0.0
    correct = 0
    total = 0
    for images, labels in tqdm(dataloader):
        images, labels = images.to(device), labels.to(device)
        latent = autoencoder.encoder(images)
        outputs = classifier(latent)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)
        _, predicted = torch.max(outputs.data,1)
        total += labels.size(0)
        correct += (predicted==labels).sum().item()

    epoch_loss = running_loss / len(dataset)
    accuracy = 100*correct/total
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}, Accuracy: {accuracy:.2f}%")

Training classifier with encoder fine-tuning...


100%|██████████| 28/28 [00:12<00:00,  2.24it/s]


Epoch [1/100], Loss: 2.1346, Accuracy: 20.34%


100%|██████████| 28/28 [00:12<00:00,  2.18it/s]


Epoch [2/100], Loss: 1.8070, Accuracy: 22.94%


...

Epoch [98/100], Loss: 1.1529, Accuracy: 57.40%


100%|██████████| 28/28 [00:13<00:00,  2.14it/s]


Epoch [99/100], Loss: 1.1456, Accuracy: 59.32%


100%|██████████| 28/28 [00:12<00:00,  2.18it/s]

Epoch [100/100], Loss: 1.1597, Accuracy: 57.63%


In [24]:
autoencoder.eval()
classifier.eval()

images, labels = next(iter(dataloader))
images = images.to(device)
latent = autoencoder.encoder(images)
outputs = classifier(latent)
_, preds = torch.max(outputs,1)
print("Predicted classes:", preds.cpu().numpy())
print("True classes:", labels.numpy())

Predicted classes: [2 2 0 1 0 0 0 1 1 2 2 5 0 1 5 2 0 0 0 2 2 5 5 2 3 2 1 1 5 0 0 2]
True classes: [2 2 1 4 0 0 3 1 1 0 0 5 0 1 5 0 1 0 1 2 2 5 5 2 4 0 4 1 5 0 0 5]
